### Authenticate
This is going to have to change for prod since we won't have access to user passwords

In [ ]:
import { authenticate } from './helpers/authentication.ts'
import { GATEWAY } from './helpers/routes.ts'

const adminToken = await authenticate(GATEWAY, 'j.campbell', 'test')

// TODO in dev only


### Register API client
For prod, best option will probably be to get a client created for us by the local team and just get the clientId and Secret and store them as github secrets and pass them in as deno args.

In [ ]:
import { registerSystem } from './helpers/gqlHandlers.ts'
// TODO in dev only
const systemRegistration = await registerSystem(adminToken)
// Get secret and id from env vars
const clientId = systemRegistration.data.registerSystem.system.clientId
const clientSecret = systemRegistration.data.registerSystem.clientSecret


In [ ]:
async function getTokenForSystemClient(
  clientId: string,
  clientSecret: string
): Promise<string> {
  const authenticateResponse = await fetch(
    `${GATEWAY}/auth/token?client_id=${clientId}&client_secret=${clientSecret}&grant_type=client_credentials`,
    {
      method: 'POST',
      headers: {
        'Content-Type': 'application/json',
        'x-correlation-id': clientId + '-' + Date.now(),
      },
    }
  )
  const res = await authenticateResponse.json()

  return res.token || res.access_token
}

const sysToken = await getTokenForSystemClient(clientId, clientSecret)
sysToken


### Fetch birth and death events

In [ ]:
import { fetchEvents } from './helpers/formsHandlers.ts'
import { extractFieldType } from './helpers/utils.ts'

const events = await fetchEvents(sysToken)

const ignoredFields = ['DIVIDER', 'PARAGRAPH']

const birthEvent = events.find((x) => x.id === 'birth')
const deathEvent = events.find((x) => x.id === 'death')

const birthEventFields = new Set(
  extractFieldType(birthEvent, 'fields')
    .filter((x) => !ignoredFields.includes(x.type))
    .map((f) => f.id)
    .filter((x) => x)
)

const deathEventFields = new Set(
  extractFieldType(deathEvent, 'fields')
    .filter((x) => !ignoredFields.includes(x.type))
    .map((f) => f.id)
    .filter((x) => x)
)


### Get all potential resolvers
Use only resolvers for used event fields to avoid nulls

In [ ]:
import defaultResolvers from './helpers/defaultResolvers.ts'
import { countryResolver } from './helpers/countryResolvers.ts'

const allResolvers = { ...defaultResolvers, ...countryResolver }

const birthResolver = Object.fromEntries(
  Object.entries(allResolvers).filter(([key, _value]) =>
    [...birthEventFields].includes(key)
  )
)

const deathResolver = Object.fromEntries(
  Object.entries(allResolvers).filter(([key, _value]) =>
    [...deathEventFields].includes(key)
  )
)

### Migrate births

In [ ]:
import {
  fetchBirthRegistration,
  declareEvent,
  fetchAllBirthRegistrations,
} from './helpers/gqlHandlers.ts'
import { transform } from './helpers/transform.ts'

const saved = []

const migrateBirth = async (entryId) => {
  // Get entry
  const birthRegistrationData = await fetchBirthRegistration(entryId, sysToken)
  if (!birthRegistrationData.data.fetchBirthRegistration) {
    console.error(JSON.stringify(birthRegistrationData, null, 2))
    throw new Error(`No data for ${entryId}`)
  }
  let document
  try {
    //transform
    document = transform(
      birthRegistrationData.data.fetchBirthRegistration,
      birthResolver
    )
    //validate
    //post

    const result = await declareEvent(document, sysToken)

    //throw if it fails?

    return result.result.data.json.id
  } catch (err) {
    console.log('document')
    console.log(JSON.stringify(document, null, 2))
    console.log('birthRegistrationData')
    console.log(JSON.stringify(birthRegistrationData))
    throw err.message
  }
}

const pageSize = 4
let itemsRemaining = 0
let page = 1

do {  
  const birthRegistrations = await fetchAllBirthRegistrations(sysToken, page, pageSize)

  const birthIds = birthRegistrations.data.searchEvents.results.map((x) => x.id)

  for (const id of birthIds) {
    const savedId = await migrateBirth(id)
    saved.push(savedId)
  }

  const totalItems = birthRegistrations.data.searchEvents.totalItems
  itemsRemaining = totalItems - (page * pageSize)
  console.log(
  `Processed ${page * pageSize} of ${totalItems} birth registrations... With ${itemsRemaining} remaining.`
  )
  page += 1
} while (itemsRemaining > 0)



### Migrate Deaths

In [ ]:
import { fetchDeathRegistration, fetchAllDeathRegistrations } from './helpers/gqlHandlers.ts'

const migrateDeath = async (entryId) => {
  // Get entry
  const deathRegistrationData = await fetchDeathRegistration(entryId, sysToken)
  //transform
  const document = transform(
    deathRegistrationData.data.fetchDeathRegistration,
    deathResolver
  )
  //validate
  //post
  const result = await declareEvent(document, sysToken)
  //throw if it fails?

  return result.result.data.json.id
}

// TODO HAS_DEATHS env var check here

const deathRegistrations = await fetchAllDeathRegistrations(sysToken, page, pageSize)
const deathIds = deathRegistrations.data.searchEvents.results.map((x) => x.id)
deathIds

for (const id of deathIds) {
  const savedId = await migrateDeath(id)
  saved.push(savedId)
}


### Output results
Could make these better to make testing easier

In [ ]:
import { REGISTER_APP } from './helpers/routes.ts'

console.log('🍞 Declarations succesfully migrated:')
console.log()
saved.forEach((s) => {
  console.log(
    `You can view the original record [here](${REGISTER_APP}/${s}/viewRecord?V2_EVENTS=false)`
  )
  console.log(
    `and the new migrated record [here](${REGISTER_APP}/events/view/${s}?V2_EVENTS=true)`
  )
})
